# <u>UNHAS: Pixel drill using interactive widgets</u>

**Contents**

  - [Overview](#Overview)
  - [Notebook setup](#Notebook-setup)
  - [Loading up the Sentinel-2 data](#Loading-up-the-Sentinel-2-data)
    - [DataCube query](#DataCube-query)
    - [Load and pre-process the data](#Load-and-pre-process-the-data)
  - [Interactive widget plots](#Interactive-widget-plots)
    - [Selecting the pixel drill location](#Selecting-the-pixel-drill-location)
    - [Pixel drill plot](#Pixel-drill-plot)
    - [Plot the selected image](#Plot-the-selected-image)

# Overview

This notebook demonstrates the use of interactive widgets in a Jupyter notebook. We will load up a few time slices of Sentinel-2 data over the Lake Tempe region, Indonesia, and display a widget allowing the user to interactively select a pixel of interest over the selected area. The time series of remote sensing values for that pixel is then plotted, allowing the user to select a given time index to use for the final display of the Sentinel-2 image acquired at that specific date.

This notebook is adapted from a [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks) example by Claire Krause.

**A note regarding compatibility**

As stated above, this notebook makes use of a specific dataset of remote sensing data, and is applied over a given region of interest. The code below will thus not run properly if the EASI deployment used to run this notebook does not also contain a similar dataset.

Users can nonetheless investigate the outputs provided in this demonstration notebook, and also modify certain variables in the code below to allow the notebook to run with a different EASI deployment (e.g. different remote sensing data, region of interest, etc.)

# Notebook setup

In a typical notebook, we would use the built-in "magic" command `%matplotlib inline` to set up the inline `matplotlib` backend in the notebook. Because we will here make use of interactive widgets, we need to invoke a different command to use the `widget` GUI backend, as done below:

In [ ]:
%matplotlib widget

Then in the cell below, we import the usual Python modules and functions needed in the rest of this notebook. Subsequently, we also open a connection to the EASI datacube.

In [ ]:
### System
import os, sys

### Datacube 
from datacube import Datacube
from datacube.utils import masking

### Data tools
import matplotlib.dates
import numpy as np

### Plotting
import ipywidgets as widgets
import matplotlib.pyplot as plt

### EASI tools
sys.path.append('/home/jovyan/git_hub_notebook/scripts/')   
   # 'scripts' available by cloning this repository: https://dev.azure.com/csiro-easi/easi-hub-public/_git/hub-notebooks
import notebook_utils

In [ ]:
dc = Datacube(app='pixel_drill')


# Loading up the Sentinel-2 data

## DataCube query

In this notebook, we're interested in a dataset of Sentinel-2 imagery (product labelled `s2_l2a` on the current EASI deployment) centred over the Lake Tempe region. We set up the corresponding spatial extent in the query dicitionary below, together with a (small) temporal window.

We will use the Sentinel-2 data for false-colour plots of the SWIR, NIR and green bands (in the corresponding RGB channels), so we add these bands to the list of measurements to load up. For the purpose of this demonstration, we will not carry out the typical data clean-up process to mask pixels affected by data quality issues (saturation, clouds, etc.) &ndash; thus, we do not include the `SCL / mask / qa` band here which would otherwise be needed for this QA filtering process.

In addition, we don't really care about the projection of the data in the plots further below. So to avoid unnecessary (and time consuming) re-projection of the data, we will here load up the Sentinel-2 dataset in its native projection. In the code below, we use the function `mostcommon_crs()` to work out the dataset's native coordinate reference system (CRS) and set the query parameter `output_crs` accordingly.

In [ ]:
product = 's2_l2a'     # Sentinel-2 data on current EASI deployment

query = { 'product': product,
          'lat': (-3.89, -4.29),           # Lake Tempe
          'lon': (119.75, 120.15),
          'measurements': ['green', 'nir_1', 'swir_2'],
          'time': ('2020-01-01', '2020-06-01'),
          'group_by': 'solar_day' }   # scene ordering

### Dataset's native projection
native_crs = notebook_utils.mostcommon_crs(dc, query)
print(f"The dataset's native CRS is: {native_crs}")

query.update({ 'output_crs': native_crs,    # EPSG code
               'resolution': (-30, 30) })   # target resolution

## Load and pre-process the data

In [ ]:
data = dc.load(**query)
data

Because our dataset is here relatively small, we did not make use of `Dask` (distributed `Xarray` processing on compute clusters) &ndash; the data is thus loaded into the system memory. As usual, we can check the size of the data object we just loaded up: without `Dask`, using a very large dataset could lead to technical and/or computational issues in the code further below in this notebook.

In [ ]:
print(f"The size of 'data' in memory is {data.nbytes/(1024**2):.2f} MB.")

Next, we can do some basic data cleaning / re-formatting to allow for a useful display of the plots further below:

In [ ]:
### Set all 'nodata' pixels to 'NaN'
data = masking.mask_invalid_data(data)

### Set pixels to 'NaN' outside valid range for Sentinel-2 data (GA S2 products)
data = data.where((data>=0) & (data<=10000))

# Interactive widget plots

## Selecting the pixel drill location

In the next cell, we (manually) select a time slice from the Sentinel-2 time series, and plot it with a widget allowing the user to click on the plot and select a desired location for the pixel drill.

First, we define the callback function `onclick` that specifies what happens when the plot is clicked &ndash; this function here updates the text of the widget `wid` and sets the (global) parameters `pixelx` and `pixely` for the pixel drill location.

Subsequently, the plot is displayed and `mpl_connect` is used to bind the `onclick` callback function to the mouse click event.

In [ ]:
### Callback function
def onclick(event):
    global pixelx, pixely
    pixelx, pixely = int(event.xdata), int(event.ydata)
    wid.value = f'pixelx : {pixelx}, pixely : {pixely}'

### Display one of the time slices (manually selected)
image_array = data[['swir_2', 'nir_1', 'green']].isel(time=1).to_array()
image_array.plot.imshow(robust=True, figsize=(8,8))

### Widget handling
fig = plt.gcf()
wid = widgets.HTML("Click on the pixel you would like to interrogate")
cid = fig.canvas.mpl_connect('button_press_event', onclick)
display(wid)

## Pixel drill plot

Once the pixel drill location is selected, the next cell plots the complete timeseries of data for that pixel. 

For the sake of this demonstration, we here simply plot the `green` spectral band values &ndash; this workflow could easily be updated to display some band arithmetic index, such as NDVI for instance.

And similar to the previous example, we make use of the `mpl_connect` function to link the (new) callback function `onclick2` to the mouse-click event on the new plot. Here, the callback function defines the global variable `selected_date` whenever the user clicks on a point on the plot.

In [ ]:
### Extract the pixel drill data
pix_drill_data = data.green.sel(y=pixely, x=pixelx, method='nearest')

### Plot the pixel drill time series
fig = plt.figure(figsize=(8,8))
plt.plot(data.time, pix_drill_data, 'ro-')
plt.xlabel('date'); plt.title('pixel drill data')
plt.ylabel('surface reflectance (green band)')

### Callback function
def onclick2(event):
    global selected_date
    tmp = event.xdata
    selected_date = str( matplotlib.dates.num2date(tmp).date() )   # convert the returned (clicked) integer to datetime string
    wid2.value = f'Date of interest : {selected_date}'

### Widget handling
wid2 = widgets.HTML("Click on a point to select the date of interest")
cid = fig.canvas.mpl_connect('button_press_event', onclick2)
display(wid2)

Depending on the selected pixel drill location, and as we have not filtered out clouds, you might see in the above plot a mix of high `green` values (e.g. >5,000) as well as lower values (perhaps around ~1500). If you click and select various dates in this time series, you will notice that the higher values correspond to images with clouds (high reflectance) covering the pixel drill location. The collection of lower values likely represent the scenes where the sensor provides a clear (cloud-free) image of the selected pixel, and so represent the `green` values of the actual ground.

## Plot the selected image

Finally, we can simply extract the selected time slice from the Sentinel-2 time series and plot it.

In [ ]:
### Convert selected date to datetime format
time_slice = np.datetime64(selected_date, 'ns')

### Use time stamp to extract and display the selected image
image_array = data[['swir_2', 'nir_1', 'green']].sel(time=time_slice, method='nearest').to_array()
image_array.plot.imshow(robust=True, figsize=(8,8));

In [ ]:
### End notebook